In [ ]:
%reset

In [ ]:
from numpy import median

import os, sys, random

import pandas as pd
import numpy as np
import swifter

from datetime import timedelta, date
from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

os.chdir('/home/adam/files/causal_inference')

from causal_inference.create_experiment.create_observations import create_observations
from causal_inference.create_experiment.create_covariates import add_covariates
from causal_inference.create_experiment.create_control import create_control_observations
from causal_inference.create_experiment.create_outcome import add_outcome
from causal_inference.create_experiment.create_medications import get_medications
from causal_inference.create_experiment.utils import add_pf_ratio

In [ ]:
# Reloads packages
os.chdir('/home/adam/files/causal_inference')
os.getcwd()

reload(sys.modules['causal_inference.create_experiment.utils'])
reload(sys.modules['causal_inference.create_experiment.create_treatment'])
reload(sys.modules['causal_inference.create_experiment.create_observations'])
reload(sys.modules['causal_inference.create_experiment.create_covariates'])
reload(sys.modules['causal_inference.create_experiment.create_control'])
reload(sys.modules['causal_inference.create_experiment.create_outcome'])
reload(sys.modules['causal_inference.create_experiment.create_medications'])


from causal_inference.create_experiment.create_observations import create_observations
from causal_inference.create_experiment.create_covariates import add_covariates
from causal_inference.create_experiment.create_control import create_control_observations
from causal_inference.create_experiment.create_outcome import add_outcome
from causal_inference.create_experiment.create_medications import get_medications
from causal_inference.create_experiment.utils import add_pf_ratio

### 1. Load observations

Loads all proning sessions that matches a 'hash_patient_id' from Patients table in the
Data Warehouse.

In [ ]:
dl = DataLoader()

In [ ]:
df = create_observations(dl,5)

In [ ]:
df = add_outcome(dl,df,2,4)

In [ ]:
df = add_outcome(dl,df,6,8)

In [ ]:
df = add_outcome(dl,df,12,16)

In [ ]:
df, _ = add_covariates(dl, df, 1000, 0, covariate_type='bmi')

In [ ]:
df, _ = add_covariates(dl, df, 24, 0, covariate_type='lab_values')

In [ ]:
df, _ = add_covariates(dl, df, 8, 0, covariate_type='vital_signs')

In [ ]:
df, _ = add_covariates(dl, df, 8, 0, covariate_type='ventilator_values')

In [ ]:
df, _ = add_covariates(dl, df, 4, 0, covariate_type='central_line')

In [ ]:
df, _ = add_covariates(dl, df, 4, 0, covariate_type='saturation')

In [ ]:
df, _ = add_covariates(dl, df, 4, 0, covariate_type='blood_gas')

In [ ]:
df = add_pf_ratio(df)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#os.chdir('/home/adam/files/data')
#df.to_csv('test', index=False)

In [ ]:
os.chdir('/home/adam/files/data')
df = pd.read_csv('test')
df.start_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.end_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.info()

### Table 1

In [ ]:
print(len(dl.get_patients(columns=['hash_patient_id']).index))
print(df.hash_patient_id.nunique())
print(df[df.treated].hash_patient_id.nunique())


### Table 2

In [ ]:
print(len(df.index))
print(len(df[df.treated].index))
print(len(df[~df.treated].index))
print(len(df[df.artificial_session == True].index))

#### Table 3

In [ ]:
df_treated = df.loc[df.treated]
print(len(df_treated[~(df_treated.peep.isna())]))
mask = ~(df_treated.peep.isna()) &\
       ~(df_treated.po2_arterial.isna()) &\
       ~(df_treated.fio2.isna())

print(len(df_treated[mask].index))

In [ ]:
df_control = df.loc[~df.treated]
print(len(df_control.index))
print(len(df_control[~(df_control.peep.isna())]))
mask = ~(df_control.peep.isna()) &\
       ~(df_control.po2_arterial.isna()) &\
       ~(df_control.fio2.isna())

print(len(df_control[mask].index))

In [ ]:
df_control = df.loc[~df.treated]
print(len(df_control.index))
print(len(df_control[~(df_control.peep.isna())]))
mask = ~(df_control.peep.isna()) &\
       ~(df_control.po2_arterial.isna()) &\
       ~(df_control.fio2.isna())

print(len(df_control[mask].index))

#### Table 4

In [ ]:
is_na = ~(df.peep.isna()) &\
       ~(df.po2_arterial.isna()) &\
       ~(df.fio2.isna())
df = df[is_na]

df['pf_ratio'] = df['po2_arterial'] / df['fio2']
df['pf_ratio'] = df['pf_ratio'].map(lambda x: round(x * 100))

In [ ]:
df.rename(columns={'po2_arterial': 'po2_arterial_inclusion_4h',
                   'peep': 'peep_inclusion_4h',
                   'fio2': 'fio2_inclusion_4h',
                   'pf_ratio_4h': 'pf_ratio_inclusion_4h'},
          inplace=True)

In [ ]:
df.treated.value_counts()

In [ ]:
inclusion = (df.pf_ratio_inclusion_4h < 150) &\
            (df.peep_inclusion_4h >= 5) &\
            (df.fio2_inclusion_4h >= 0.6)

df_inclusion = df[inclusion]

df_inclusion.treated.value_counts()

In [ ]:
os.chdir('/home/adam/files/data')
df_inclusion.to_csv('data_skeleton_4h_only_included.csv', index=False)

### Add outcomes

In [ ]:
df_inclusion.info()

In [ ]:
df = add_outcome(dl, df, 2, 4)

df.info()

In [ ]:
df.head()

In [ ]:
df.loc[:, ['start_timestamp', 'end_timestamp']].min(axis=1)

In [ ]:
df['pf_ratio_4h_outcome_time_until_interval_end'].dtype

In [ ]:
df_covariates, report = add_covariates(dl, df, interval_start=12, interval_end=0)

In [ ]:
df_covariates.info()

### NEW

In [ ]:
# Distribution of proning sessions in hospitals

df_treated.pacmed_origin_hospital.value_counts()

In [ ]:
#Distribution of outcomes

df_treated.outcome.value_counts()

In [ ]:
# has died during session

df_treated.has_died_during_session.value_counts()

### 3. Loading inclusion criteria

In [ ]:
df_treated, time_report = add_covariates(dl=dl,
                                         df=df_treated,
                                         interval_start=4,
                                         interval_end=0,
                                         covariates=['fio2',
                                                     'peep',
                                                     'po2_arterial']
                                         )

In [ ]:
df_treated_6, time_report_6 = add_covariates(dl=dl,
                                         df=df_treated,
                                         interval_start=6,
                                         interval_end=0,
                                         covariates=['fio2',
                                                     'peep',
                                                     'po2_arterial']
                                         )

In [ ]:
# rename columns
df_treated_6.rename(columns={"fio2_x": "fio2_4h",
                             "peep_x": "peep_4h",
                             "po2_arterial_x": "po2_arterial_4h",
                             "fio2_y": "fio2_6h",
                             "peep_y": "peep_6h",
                             "po2_arterial_y": "po2_arterial_6h"},
                    inplace=True)
df_treated_6.info()

In [ ]:
os.chdir('/home/adam/files/data')

df_treated_6.to_csv('treated_data_skeleton_6h_inclusion.csv', index=False)
time_report_6.to_csv('treated_data_skeleton_6h_inclusion_time_report.csv', index=True)

### 4. Analysis

In [ ]:
os.chdir('/home/adam/files/data')
df = pd.read_csv('treated_data_skeleton_6h_inclusion.csv')
df.start_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.end_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.info()

In [ ]:
df = df.dropna(subset=['fio2_4h', 'peep_4h', 'po2_arterial_4h'])

In [ ]:
df.info()

In [ ]:
# add p_f ratio

df.loc[:, 'pf_4h'] = df['po2_arterial_4h'] / df['fio2_4h']
df.loc[:, 'pf_4h'] = df.pf_4h.map(lambda x: round(x * 100))

In [ ]:
df.head()

#### Proning Length w.r.t. hospitals

In [ ]:
df.pacmed_origin_hospital.value_counts()

In [ ]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=df.duration_hours.values,
                            bins='auto',
                            color='#0504aa',
                            alpha=0.7,
                            rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('My Very Own Histogram')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

In [ ]:
import seaborn as sns

hospitals = ['amc', 'vumc', 'olvg', 'erasmus', 'amphia']
df_hospitals = df[df.pacmed_origin_hospital.isin(hospitals)]

sns.displot(data= df_hospitals,
            x="duration_hours",
            hue='pacmed_origin_hospital',
            kind="kde",
            bw_adjust=.7,
            cut = 0)

#### Distribution of parameters w.r.t. hospitals

In [ ]:
df_plot = df_hospitals[df.pf_4h < 300]

sns.displot(data=df_plot,
            x="pf_4h",
            hue='pacmed_origin_hospital',
            element="step",
            bins=10)

In [ ]:
sns.displot(data=df[df.pf_4h < 300],
            x="pf_4h",
            element="step",
            bins=10)

#### Creating control


In [ ]:
df.head()

In [ ]:
df_control = df.loc[(df["treated"] == False)]
df_control.head()

In [ ]:
# 1. Ignore control less than 8

df_1 = df_control.loc[(df_control.duration_hours >= 8)]
df_1.head()

In [ ]:
# 2. Load measurements if the sessions lasts for 8 hours. This is ready
# to load blood gas

df_2 =

In [ ]:
# 3. Data to be split is prepared

df_3 = df_1.loc[(df_1.duration_hours > 8)]


start = df_3.start_timestamp
end = df_3.start_timestamp + df_control.duration_hours - 8

n_of_measuremnts?
# 2. Round to half hour

# 3.

In [ ]:
df_sample = df_control[df_control.duration_hours > 8].sample(3)

In [ ]:
sample = create_control_observations(dl, df_sample)

In [ ]:
sample



In [ ]:
df_1 = pd.DataFrame([])
df_2 = pd.DataFrame([])

pd.concat([df_1, df_2])

In [ ]:
df_medications = get_medications(dl)

In [ ]:
df_medications.pacmed_name.value_counts()

In [ ]:
df_medications.pacmed_subname.value_counts()

In [ ]:
df_medications = dl.get_medications(columns=['hash_patient_id',
                                                 'pacmed_name',
                                                 'pacmed_subname',
                                                 'start_timestamp',
                                                 'end_timestamp',
                                                 'total_dose',
                                                 'dose_unit_name'])

In [ ]:
df_medications[df_medications.pacmed_subname == 'med_vitamin_k_antagonists']

In [ ]:
df.info()

In [ ]:
df.loc[df.treated, 'duration_hours'].describe().round()

In [ ]:
mask = ~(df['po2_arterial_inclusion_4h'].isna()) & ~(df['fio2_inclusion_4h'].isna())
df.loc[mask, 'pf_ratio_inclusion_4h'] = df.loc[mask, 'po2_arterial_inclusion_4h'] / df.loc[mask, 'fio2_inclusion_4h']
df['pf_ratio_inclusion_4h'].describe()

In [ ]:
df.info()

In [ ]:
len(df.dropna(subset=[pco2_arterial
po2_arterial
all pacmed_names starting with bicarbonate_
all pacmed_names starting with ph_
all pacmed_names starting with lactate_ EXCEPT for lactate_dehydrogenase
so2_central_venous
o2_saturation]).index) / len(df.index)